In [13]:
import os
import vertexai
from typing import List, Dict, Any, Optional
from pathlib import Path
from dotenv import load_dotenv

env_path = Path.cwd().parent / '.env'
load_dotenv(env_path)

GCP_PROJECT = os.getenv("GCP_PROJECT")
GCP_REGION = os.getenv("GCP_REGION")
GOOGLE_MAP_KEY = os.getenv("GOOGLE_MAP_KEY")

vertexai.init(project=GCP_PROJECT, location=GCP_REGION)

In [14]:
from agent_tools import get_lat_lon_from_address, get_weather_forecast

def get_weather(address: str) -> Optional[List[Dict[str, Any]]]:
    """Takes an address and returns a weather forcast from National Weather Service.

    Args:
        address: The street address or place name to geocode (e.g., "1600 Amphitheatre Parkway, Mountain View, CA").

    Returns:
        A list of dictionaries, where each dictionary represents a forecast
        period (e.g., 'Tonight', 'Thursday'). Returns None if an error occurs.
    """
    try:
        lat, lon = get_lat_lon_from_address(address=address, api_key=GOOGLE_MAP_KEY)
        forecast = get_weather_forecast(lat, lon)
        return forecast
    except Exception as e:
        print(f"Something broke. Good luck fixing:\n{e}")
        return None
    

In [17]:
from google.adk.agents import Agent
from vertexai.preview import reasoning_engines
from IPython.display import Markdown, display

weather_agent = Agent(
    name="weather_agent_v1",
    model="gemini-2.0-flash", # Can be a string for Gemini or a LiteLlm object
    description="Provides weather information for specific cities.",
    instruction="You are a helpful weather assistant. "
                "When the user asks for the weather in a specific city, "
                "use the 'get_weather' tool to find the information. "
                "If the tool returns an error, inform the user politely. "
                "If the tool is successful, present the weather report clearly.",
    tools=[get_weather], 
)


app = reasoning_engines.AdkApp(
    agent=weather_agent,
    enable_tracing=False,
)

user_id = "test-user-id"
session = app.create_session(user_id=user_id)

for event in app.stream_query(
    user_id=user_id,
    session_id = session.id,
    message="What's the weather in New York, Seattle, and Chicago for the next three days? Provide response as markdown table.",
):
  lastevent = event

display(Markdown(lastevent["content"]["parts"][0]["text"]))


Here's a weather forecast for the next three days for New York, Seattle, and Chicago.

| City    | Day     | Forecast                                                                                                                                                                           | Temperature |
| :------ | :------ | :--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :---------- |
| New York | Today   | Partly sunny, with a high near 85. A chance of showers and thunderstorms between 2pm and 5pm, then a slight chance of showers and thunderstorms.                                  | 85°F       |
|         | Tonight | Mostly cloudy. Low around 76, with temperatures rising to around 78 overnight. A chance of showers and thunderstorms before 2am, then showers and thunderstorms likely.            | 76°F       |
|         | Thursday| Mostly cloudy. High near 79, with temperatures falling to around 77 in the afternoon. Showers and thunderstorms likely.                                                           | 79°F       |
| Seattle | Today   | Mostly cloudy. High near 71, with temperatures falling to around 69 in the afternoon. A chance of rain.                                                                            | 71°F       |
|         | Tonight | Cloudy, with a low around 57. A chance of rain before 11pm.                                                                                                                         | 57°F       |
|         | Thursday| Partly sunny, with a high near 74.                                                                                                                                                  | 74°F       |
| Chicago | Today   | Mostly sunny. High near 78, with temperatures falling to around 74 in the afternoon. Isolated showers and thunderstorms between 1pm and 4pm.                                       | 78°F       |
|         | Tonight | Partly cloudy, with a low around 69.                                                                                                                                                | 69°F       |
|         | Thursday| Mostly sunny, with a high near 78.                                                                                                                                                  | 78°F       |
